### With predicted rental prices, we are now interested in the growth rates for each SA2

In [22]:
import pandas as pd
import glob
import os

#path = r'../data/curated/2023_2027_rental_prediction_lr/'
#path = r'../data/curated/2023_2027_rental_prediction_xgboost/' # use your path
path = r'../data/curated/2023_2027_rental_prediction_rf/'
all_files = glob.glob(os.path.join(path , "*.csv"))

li = []

for filename in sorted(all_files):
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

merged_df = pd.concat(li, axis=0, ignore_index=True)
predicted = merged_df.groupby(['year', 'sa2_2021'], as_index=False).agg({'predicted_price': 'mean'})
predicted

,year,sa2_2021,predicted_price
0,2023,201011001,426.780607
1,2023,201011002,453.324108
2,2023,201011005,371.814574
3,2023,201011006,459.171143
4,2023,201011007,478.270151
...,...,...,...
2490,2027,217031476,528.641853
2491,2027,217041477,423.750503
2492,2027,217041478,424.167161
2493,2027,217041479,419.449507


In [23]:
rental_2022 = pd.read_csv('../data/curated/merged_dataset/2022_merged_data.csv')
rental_2022.dropna(inplace=True)
rental_2022 = rental_2022.groupby(['year', 'sa2_2021'], as_index=False).agg({'weekly_rent': 'mean'})
rental_2022.rename({'weekly_rent': 'predicted_price'}, axis=1, inplace=True)
rental_2022
#rental_2022[rental_2022['sa2_2021']==206041127]
#set(rental_2022['sa2_2021'].unique()) - set(predicted['sa2_2021'].unique())

,year,sa2_2021,predicted_price
0,2022,201011001,437.468013
1,2022,201011002,421.270053
2,2022,201011005,402.559524
3,2022,201011006,440.000000
4,2022,201011007,530.625000
...,...,...,...
494,2022,217031476,518.750000
495,2022,217041477,414.069767
496,2022,217041478,771.301370
497,2022,217041479,437.025641


In [24]:
growth_df = pd.concat([rental_2022, predicted], axis=0).reset_index(drop=True)
#df[df['sa2_2021']==201011001]
growth_df

,year,sa2_2021,predicted_price
0,2022,201011001,437.468013
1,2022,201011002,421.270053
2,2022,201011005,402.559524
3,2022,201011006,440.000000
4,2022,201011007,530.625000
...,...,...,...
2989,2027,217031476,528.641853
2990,2027,217041477,423.750503
2991,2027,217041478,424.167161
2992,2027,217041479,419.449507


In [25]:
growth_dict = dict()
for sa2 in growth_df['sa2_2021'].unique():
    growth_dict[sa2] = []
    subset = growth_df[growth_df['sa2_2021']==sa2]
    for i in range(len(subset['predicted_price'])-1):
        # (current year price - past year price) / past year price
        growth_rate = (subset.iloc[i+1]['predicted_price'] - subset.iloc[i]['predicted_price'])/subset.iloc[i]['predicted_price']
        growth_dict[sa2].append(growth_rate)


In [26]:
growth_dict

{201011001: [-0.024430144818526123,
  0.020597532297906403,
  0.005660105434552479,
  -0.0034733369775385775,
  0.004254549315966025],
 201011002: [0.0760890882062723,
  0.004398712373320705,
  0.0012433695187468552,
  0.0035053656033821263,
  0.05283560685406114],
 201011005: [-0.07637367387231803,
  0.0037911736124045738,
  0.00027465225184659996,
  0.003394080142125095,
  -0.00044662773363725864],
 201011006: [0.043570778766820306,
  -0.06266737578247894,
  5.4683084769226264e-05,
  0.005729124066946333,
  -0.000529811196217042],
 201011007: [-0.09866638193889347,
  -0.0422989234881849,
  0.008072074441424713,
  -0.01038261545757797,
  0.00538727338603309],
 201011008: [-0.026272495193429363,
  0.04829127821668284,
  0.03074733005463314,
  0.012983981533716825,
  -0.00022527815599982626],
 201011481: [0.03476566533653481,
  -0.0007404831646854587,
  0.010097864038600165,
  0.016619872731226157,
  0.002130128482177235],
 201011482: [0.04313268038256005,
  0.01614014299706797,
  0.026

In [27]:
import numpy as np
for key in growth_dict.keys():
    growth_dict[key] = np.mean(growth_dict[key])
growth_dict

{201011001: 0.0005217410504720413,
 201011002: 0.02761442851115663,
 201011005: -0.0138720791199158,
 201011006: -0.0027685202120320235,
 201011007: -0.0275777146114397,
 201011008: 0.013104963291120725,
 201011481: 0.01257460948477058,
 201011482: 0.0230212162324686,
 201011483: 0.017391592831614514,
 201011484: 0.02563722171268738,
 201021009: 0.04352021327630025,
 201021010: 0.019043588403288923,
 201021011: -0.030719057087298933,
 201021012: 0.02196568287311223,
 201031013: 0.02985118748288786,
 201031014: -0.0005628823024676443,
 201031015: 0.010020102658822519,
 201031016: 0.028254174888981114,
 201031017: 0.03187031796075366,
 202011018: 0.016742762914513758,
 202011019: -0.011798583445583875,
 202011020: 0.007442329825783153,
 202011021: 0.0008532170754640179,
 202011022: -0.00795033810484282,
 202011023: -0.005921031414598923,
 202011024: -0.012425649975437268,
 202011025: 0.007150252215153445,
 202021026: 0.015207787976694531,
 202021027: 0.004860702039192292,
 202021028: -0.

In [28]:
# Sort dictionary so that the dictionary values are sorted as descneding order 
# The first subrub in the dictionary = the suburb which is predicted to have the highest average growth rate in the next 5 years 
sorted_dict = dict(sorted(growth_dict.items(), key=lambda item: item[1], reverse=True))
sorted_dict_list = list(sorted_dict)

sa2_list = list(sorted_dict.keys())
growth_list = list(sorted_dict.values())

top_ten_names = []

sa2_dict = pd.read_csv("../data/curated/sa2_vic_2021.csv")

i=0
while i < len(sa2_list):
    selected = sa2_list[i]
    suburb_name = sa2_dict[sa2_dict['SA2_CODE21']==selected]['SA2_NAME21'].to_list()[0]
    top_ten_names.append(suburb_name)
    i+=1

# Make a dataframe with the prediced results with columns of suburb name, its sa2 code and its growth rate
# new_csv_name = "../data/curated/final_growing_rates_lr.csv" 
# new_csv_name = "../data/curated/final_growing_rates_xgboost.csv" 
new_csv_name = "../data/curated/final_growing_rates_rf.csv" 

data = {'Suburb_Name': top_ten_names,
        'sa2_2021': sa2_list,
        'Growth_Rate': growth_list}

df = pd.DataFrame(data)
print(df)
df.to_csv(new_csv_name, index=False)

            Suburb_Name   sa2_2021  Growth_Rate
0    Narre Warren North  212021299     0.147462
1     Point Cook - East  213051464     0.141579
2                Elwood  206051129     0.101565
3          Yarriambiack  215011394     0.099641
4          Nhill Region  215011390     0.089190
..                  ...        ...          ...
494     Bundoora - East  209011196    -0.068789
495      Sunshine North  213011337    -0.072544
496              Echuca  216011406    -0.088702
497    Mansfield (Vic.)  204011057    -0.089870
498        Moyne - West  217041478    -0.099065

[499 rows x 3 columns]
